## Raod Map

We want to:

- [x] plot the ACFs for each **ensAvg** **property** individually as a **subplot** in a **space** **figure**



## General Settings

### Importing packages

In [ ]:
# Importing necessary packages:
#import re
#import os
from glob import glob
#import math
import numpy as np
import pandas as pd
#import datetime as dt
#import itertools

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl
from matplotlib import rcParams, cycler
import matplotlib.lines as lines
from collections import OrderedDict
from matplotlib.legend import Legend
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from matplotlib.collections import PolyCollection # for chaing confidence interval color in statsmodels.graphics.tsaplots.plot_acf

import seaborn as sns

import statsmodels.tsa.stattools as tsas
import statsmodels.graphics.tsaplots as tsap

from polyphys.visualize import plotter
from polyphys.visualize import tuner
from polyphys.manage.parser import SumRule, TransFoci
from polyphys.manage import organizer
from polyphys.analyze import measurer
from polyphys.manage import utilizer

from polyphys.analyze import correlations 
from scipy import optimize

# Project details

project_details ={
    'SumRule':{
        'parser': SumRule,
        'space_pat': 'N*D*ac*',
        'hierarchy': 'N*',
        'space_hierarchy': 'N*',
        'attributes': ['space', 'ensemble_long', 'ensemble', 'nmon', 'dcyl',
                       'dcrowd','phi_c_bulk'
                      ],
        'time_varying_props': [ 'asphericityTMon', 'fsdTMon', 'gyrTMon',
                               'rfloryTMon','shapeTMon'],
        'equil_measures': [np.mean, np.var, measurer.sem],
        'equil_attributes': ['space', 'ensemble_long', 'ensemble', 'nmon',
                             'dcyl','dcrowd', 'phi_c_bulk', 
                             'phi_c_bulk_round'
                            ],
        'equil_properties': ['asphericityMon-mean', 'asphericityMon-var',
                             'asphericityMon-sem', 'fsdMon-mean',
                             'fsdMon-var', 'fsdMon-sem', 'gyrMon-mean',
                             'gyrMon-var', 'gyrMon-sem', 'rfloryMon-mean',
                             'rfloryMon-var', 'rfloryMon-sem',
                             'shapeMon-mean', 'shapeMon-var', 'shapeMon-sem']
    },
    'TransFoci':{
        'parser': TransFoci,
        'space_pat': 'ns*nl*al*D*ac*',
        'hierarchy': 'eps*',
        'space_hierarchy': 'ns*',
        'attributes': ['space', 'ensemble_long', 'ensemble', 'nmon_small',
                       'nmon_large','dmon_large', 'dcyl', 'dcrowd',
                       'phi_c_bulk'
                      ],
        'time_varying_props': ['asphericityTMon', 'fsdTMon', 'gyrTMon',
                               'shapeTMon'],
        'equil_measures': [np.mean, np.var, measurer.sem],
        'equil_attributes': ['ensemble_long', 'ensemble', 'space', 'dcyl',
                             'dmon_large', 'nmon_large', 'nmon_small',
                             'dcrowd', 'phi_c_bulk', 'phi_c_bulk_round'],
        'equil_properties': ['asphericityMon-mean', 'asphericityMon-var',
                             'asphericityMon-sem', 'fsdMon-mean',
                             'fsdMon-var', 'fsdMon-sem', 'gyrMon-mean',
                             'gyrMon-var', 'gyrMon-sem', 'shapeMon-mean',
                             'shapeMon-var', 'shapeMon-sem']
    }
}

# loading databases:
analysis_db = '/Users/amirhsi_mini/research_data/analysis/'
project = 'TransFoci'
phase='ensAvg'
geometry='biaxial'
group = "bug"
species = "Mon"
space_dbs = glob(analysis_db + project_details[project]['space_pat'])

# Auto-correlation functions

##### Note: In a given space, ensAvg groups differs in their values of bulk volume fraction of crowders (column name: phi_c_bulk)

### Theoretical background:

Below, all the several definitions of autocorrelation function (acf) in the literature are listed. Assuming $A(t)$ is a discrete time-varying property of interest and is collected every $M$ timesteps in a molecular dynamics simulation with the time step $\Delta t$ and time unit $\hat{t}$. First, we define the mean (or average) and unbiased variance of $A(t)$ as what follows

$$\langle A\rangle=\frac{1}{t_{max}}\sum_{t=1}^{t_{max}}A(t)$$
$$\sigma^2_A=\langle A^2 \rangle-\langle A\rangle^2=\frac{1}{t_{max}-1}\sum_{t=1}^{t_{max}}(A(t)-\langle A\rangle)^2$$

where $t_{max}$ is the total number of collected cofigurations or timesteps, or simply the size of $A(t)$, implying the total number of simulated timesteps is $t_{max}\times M$. Given this definition, the acf has been defined in the following ways in the literature:

1. Allen and Tildesley define the *non-normalized* acf in "Computer simulation of liquids - 2017" as

$$c_{AA}(h) = \frac{1}{h_{max}}\sum_{h_0=1}^{h_{max}}A(h_0)A(h_0+h)$$

where $h_{max}=\frac{t_{max}-1}{h}$

2. *1989 - Murat M Greet GS - Structure of a Grafted Polymer Brush A Molecular Dynamics Simulation* defines the acf in "Structure of a Grafted Polymer Brush A Molecular Dynamics Simulation - 1989" as 

$$c_{AA}(h) =\frac{\langle (A(h)-\langle A \rangle)(A(0)-\langle A \rangle)\rangle}{\sigma^2_A}=\frac{\langle A(h)A(0)\rangle-\langle A \rangle \langle A(0) \rangle}{\sigma^2_A}$$

See the rest of this artciel to see how correlation time is calculated. See the rest of this artciel to see how correlation time is calculated.

3. In *1989 - Grest GS Kremer K Witten TA et el - Relaxation of Self-Entangled Many-Arm Star Polymers, 1987 - Grest GS Kremer K Witten TA - Structure of Many-Arm Star Polymers A Molecular Dynamics Simulation, and 1977 - Kranbuehl DE Verdier PH - Relaxation of the aspherical shapes of random-coil polymer chains*, the acf is

$$c_{AA}(h) =\frac{\langle A(h)A(0)\rangle-\langle A \rangle^2}{\sigma^2_A}$$

where the average is performed over the initial time steps $h=h_0=0$ taken every $K$ timesteps. Kranbuehl and Verdier runs the whole simulation several times the relaxation time of the end-to-end length.

4. In *1978 - Rapaport DC - Molecular dynamics simulation of polymer chains with excluded volum, and 1979 - Bishop M Ceperley D Frisch HL - Molecular dynamics of polymeric systems* the afc is given by

$$c_{AA}(h) =\frac{\langle A(h)A(h_0+h)\rangle-\langle A(h_0) \rangle^2}{\sigma^2_A(h_0)}$$

where

$$\sigma^2_A(h_0)=\langle A(h_0)^2 \rangle-\langle A(h_0)\rangle^2$$

The averaging should be done over ensembles; however, in practice, the ensemble average is replaced by the time average provided that the timesteps or time interval between successvie $h_0$ values are sufficently lagre so the configurations are uncorrelated.

5. *1981 - Bruns W Bansal R - Molecular dynamics study of a single polymer chain in solution* defines the afc as 

$$c_{\vec{A}\vec{A}}(h) = \frac{\sum_{h_0=1}^{t_{max}-h}\vec{A}(h_0).\vec{A}(h_0+h)}{\sum_{h_0=1}^{t_{max}-h}\vec{A}^2(h_0)}$$

where $\vec{A}$ is a vector quantity and $h_0$ is summed over $1,1+p,1+2p,\dots$ instead of $h_0=1,2,3,\dots,t_{max}-h$. For a scalar quantity, this definition is used:

$$c_{AA}(h) = \frac{(t_{max}-h)\sum_{h_0=1}^{t_{max}-h}A(h_0)A(h_0+h)-\sum_{h_0=1}^{t_{max}-h}A(h_0)\sum_{h_0=1}^{t_{max}-h}A(h_0+h)}{\sqrt{(t_{max}-h)\sum_{h_0=1}^{t_{max}-h}A^2(h_0)-[\sum_{h_0=1}^{t_{max}-h}A^2(h_0)]^2}-\sqrt{(t_{max}-h)\sum_{h_0=1}^{t_{max}-h}A^2(h_0+h)-[\sum_{h_0=1}^{t_{max}-h}A^2(h_0+h)]^2}}=
\frac{\langle A(h_0)A(h_0+h)\rangle-\langle A(h_0)\rangle\langle A(h_0+h)\rangle}{\sqrt{\langle A^2(h_0)\rangle-\langle A(h_0)\rangle^2}-\sqrt{\langle A^2(h_0+h)\rangle-\langle A(h_0+h)\rangle^2}}$$

6. *Brockwell PJ Davis RA - 2016 - Introduction to Time Series and Forecasting* defines the afc in the following way

$$c_{AA}(h) = \frac{cov_{A}(h)}{\sigma_A^2} = \frac{1}{\sigma_A^2}\frac{1}{t_{max}}\sum_{h_0=1}^{t_{max}-h} [A(h_0+h)-\langle A\rangle][A(h_0)-\langle A\rangle]$$





### How to run this Notebook:

1. Check ((spaces_to_read**
2. Check **group** and **species**.
3. check **database**.
4. delete extra **physical** properties based on the preoject.
5. check **nlags** and **ticks**.
6. check **parser** and **spaces_title**.


### Grouping and filtering

Since the datasets we work with are large, it is a good idea to 
- define some filters to ease subsetting the dataframes.
- define distinguishable colors for the volume fraction of crowders as the changing parameter that defines different ensAvg groups in a space
- round the value of this changing parameter.

In [ ]:
# read data
acf_space_dbs = [space_db for space_db in space_dbs if
                 space_db.endswith('acf.parquet.brotli')
                ]
acf = [pd.read_parquet(space_db) for space_db in acf_space_dbs]
acf = pd.concat(acf,axis=0, ignore_index=True)
acf.reset_index(inplace=True,drop=True)

# a sorted list of unique spaces in the dataset
spaces = acf.loc[:,'space'].drop_duplicates().sort_values()
spaces = sorted(spaces, key = organizer.sort_by_alphanumeric)

# rounding phi_c as facgtors of 0.025:
# add rounded phi_crds to the dataset
divisor = 0.025
round_to = 3
acf['phi_c_bulk_round'] = acf['phi_c_bulk'].apply(
    utilizer.round_up_nearest, args=[divisor, round_to]
)
# phi_c to drop
acf = acf.loc[~acf['phi_c_bulk_round'].isin([0.025, 0.05, 0.075, 0.125, 0.175]),:]
acf.reset_index(inplace=True, drop=True)
# setting colors for unique crd_c
phi_crds = acf.loc[:,'phi_c_bulk_round'].unique()
phi_crds.sort()
flar_cmap = mpl.colors.ListedColormap(sns.cm._flare_lut)
flare_cmap_cut = tuner.truncated_colormap(flar_cmap,  min_value=0.0, max_value=1.0, ncolors=200)
mpl.cm.register_cmap("flare_cmap_cut", flare_cmap_cut)
phi_colors = sns.color_palette("flare_cmap_cut", len(phi_crds))#,as_cmap=True)
# add rounded phi_crds to the dataset
# Define a list of unique physical properties:
properties = [property_ for property_ in acf.columns if '-mean' in property_]
properties.sort()
# Define unique specifications for each physical property:
acf_properties_specs =  {
    'SumRule': {
        'rfloryTMon': {
            'name': 'Flory radius',
            'symbol': r'$C_{R_FR_F}(\hat{t}_{lags})$',
            'color': 'firebrick'
        },
        'gyrTMon': {
            'name': 'radius of gyration',
            'symbol': r'$C_{R_gR_g}(\hat{t}_{lags})$',
            'color':'steelblue'
                   },
        'fsdTMon': {
            'name': 'furthermost distance',
            'symbol': r'$C_{LL}(\hat{t}_{lags})$',
            'color': 'forestgreen'
        },
        'asphericityTMon': {
            'name': 'asphericity',
            'symbol': r'$C_{\Delta\Delta}(\hat{t}_{lags})$',
            'color': 'goldenrod'
        },
        'shapeTMon': {
            'name': 'shape parameter',
            'symbol': r'$C_{SS}(\hat{t}_{lags})$',
            'color': 'orchid'
        }
    },
    'TransFoci': {
        'gyrTMon': {
            'name': 'radius of gyration',
            'symbol': r'$C_{R_gR_g}(\hat{t}_{lags})$',
            'color':'steelblue'
                   },
        'fsdTMon': {
            'name': 'furthermost distance',
            'symbol': r'$C_{LL}(\hat{t}_{lags})$',
            'color': 'forestgreen'
        },
        'asphericityTMon': {
            'name': 'asphericity',
            'symbol': r'$C_{\Delta\Delta}(\hat{t}_{lags})$',
            'color': 'goldenrod'
        },
        'shapeTMon': {
            'name': 'shape parameter',
            'symbol': r'$C_{SS}(\hat{t}_{lags})$',
            'color': 'orchid'
        }
    }
}
acf_props_specs_project = acf_properties_specs[project]
space_titles = {}
for space in spaces:
    s_info = project_details[project]['parser'](
        space, 
        geometry=geometry,
        group=group,
        lineage='space',
        ispath=False
    )
    space_title_style = {
        'SumRule': fr" $N={s_info.nmon}, D={s_info.dcyl}, a_c={s_info.dcrowd}$",
        'TransFoci': fr" $n_s={s_info.nmon_small}, n_l={s_info.nmon_large}, a_l={s_info.dmon_large}, D={s_info.dcyl}, a_c={s_info.dcrowd}$"
        }
    space_titles[space] = space_title_style[project]

### Some explorations:

In [ ]:
acf.head()

In [ ]:
acf.columns

In [ ]:
acf.info()

### ACF with CIs for all the physical properties per space columns wrapped with $\phi_c^{bulk}$

In [ ]:
nlags=7000
ticks = int(nlags/7)
for space in spaces:
    space_acf = acf[acf.space==space]
    plotter.p_acf_with_ci_space(
        space_acf,
        space,
        space_titles[space],
        project,
        properties=acf_props_specs_project,
        ncols=3,
        xlimits=(0, nlags, ticks),
        ylimits=(-0.25, 1, 0.25),
        lags=nlags,
        dpi=100,
        fontsize=18
    )

### ACF of each physical property in a project collored with an attribute

In [ ]:
#property_= 'gyrTMon'
legend_anchor = (1.1,1.02)
nrows = len(spaces)
for property_ in acf_props_specs_project.keys():
    plotter.p_acf_allInOne_project(
        acf,
        project,
        space_titles,
        property_,
        acf_props_specs_project[property_],
        phi_crds,
        phi_colors,
        xlimits=(0, nlags, ticks),
        nrows=nrows,
        ncols=1,
        legend_anchor=legend_anchor,
        lags=nlags
    )  

# Chain size plots

### Grouping and filtering

Since the datasets we work with are large, it is a good idea to 
- define some filters to ease subsetting the dataframes.
- define distinguishable colors for the volume fraction of crowders as the changing parameter that defines different ensAvg groups in a space
- round the value of this changing parameter.

In [ ]:
chainsize_space_dbs = [space_db for space_db in space_dbs if
                 space_db.endswith('chainSize.parquet.brotli')
                      ]
chainsize = [pd.read_parquet(space_db) for space_db in chainsize_space_dbs]
chainsize = pd.concat(chainsize,axis=0, ignore_index=True)
chainsize.reset_index(inplace=True,drop=True)
# droping some of the columns
cols_to_drop = {
    'SumRule': ['asphericityTMon-var', 'asphericityTMon-sem', 'fsdTMon-var',
                'fsdTMon-sem', 'gyrTMon-var', 'gyrTMon-sem', 'rfloryTMon-var',
                'rfloryTMon-sem', 'shapeTMon-var', 'shapeTMon-sem'],
    'TransFoci': ['asphericityTMon-var', 'asphericityTMon-sem', 'fsdTMon-var',
                  'fsdTMon-sem', 'gyrTMon-var', 'gyrTMon-sem',
                  'shapeTMon-var','shapeTMon-sem']
}
chainsize.drop(columns=cols_to_drop[project],inplace=True)

# a sorted list of unique spaces in the dataset
spaces = chainsize.loc[:,'space'].drop_duplicates().sort_values()
spaces = sorted(spaces, key = organizer.sort_by_alphanumeric)
# rounding phi_c as facgtors of 0.025:
# add rounded phi_crds to the dataset
divisor = 0.025
round_to = 3
chainsize['phi_c_bulk_round'] = chainsize['phi_c_bulk'].apply(
    utilizer.round_up_nearest, args=[divisor, round_to]
)
# phi_c to drop
chainsize = chainsize.loc[~chainsize['phi_c_bulk_round'].isin([0.025, 0.05, 0.075, 0.125, 0.175]),:]
chainsize.reset_index(inplace=True, drop=True)
# setting colors for unique crd_c
phi_crds = chainsize.loc[:,'phi_c_bulk_round'].unique()
phi_crds.sort()
flar_cmap = mpl.colors.ListedColormap(sns.cm._flare_lut)
flare_cmap_cut = tuner.truncated_colormap(flar_cmap,  min_value=0.0, max_value=1.0, ncolors=200)
mpl.cm.register_cmap("flare_cmap_cut", flare_cmap_cut)
phi_colors = sns.color_palette("flare_cmap_cut", len(phi_crds))#,as_cmap=True)
space_colors = sns.color_palette("flare_cmap_cut", len(spaces))#,as_cmap=True)
# setting colors for unique crd_c
flar_cmap = mpl.colors.ListedColormap(sns.cm._flare_lut)
flare_cmap_cut = tuner.truncated_colormap(flar_cmap,  min_value=0.0, max_value=1.0, ncolors=200)
mpl.cm.register_cmap("flare_cmap_cut", flare_cmap_cut)
phi_colors = sns.color_palette("flare_cmap_cut", len(phi_crds))#,as_cmap=True)
# add rounded phi_crds to the dataset
# Define a list of unique physical properties:
properties = [property_.split('-mean')[0] for property_ in chainsize.columns if '-mean' in property_]
# change the name of cols after fining properties based on "mean" in their name
new_columns = [property_.split('-mean')[0] for property_ in chainsize.columns]
properties.sort()
chainsize.columns = new_columns
print(properties)
# Define unique specifications for each physical property:
chainsize_properties_specs =  {
    'SumRule': {
        'rfloryTMon': {
            'name': 'Flory radius',
            'symbol': r'$R_F(\hat{t})$',
            'color': 'firebrick'
        },
        'gyrTMon': {
            'name': 'radius of gyration',
            'symbol': r'$R_g(\hat{t})$',
            'color':'steelblue'
        },
        'fsdTMon': {
            'name': 'furthermost distance',
            'symbol': r'$L(\hat{t})$',
            'color': 'forestgreen'
        },
        'asphericityTMon': {
            'name': 'asphericity',
            'symbol': r'$\Delta(\hat{t})$',
            'color': 'goldenrod'
        },
        'shapeTMon': {
            'name': 'shape parameter',
            'symbol': r'$S(\hat{t})$',
            'color': 'orchid'
        }
    },
    'TransFoci':{
        'gyrTMon': {
            'name': 'radius of gyration',
            'symbol': r'$R_g(\hat{t})$',
            'color':'steelblue'
        },
        'fsdTMon': {
            'name': 'furthermost distance',
            'symbol': r'$L(\hat{t})$',
            'color': 'forestgreen'
        },
        'asphericityTMon': {
            'name': 'asphericity',
            'symbol': r'$\Delta(\hat{t})$',
            'color': 'goldenrod'
        },
        'shapeTMon': {
            'name': 'shape parameter',
            'symbol': r'$S(\hat{t})$',
            'color': 'orchid'
        }
    }
}
chainsize_props_specs_project = chainsize_properties_specs[project]
attr_titles = {
    "phi_c_bulk_round": r"$\phi_c$",
    "time": "$\hat{{t}}$",
    "lags": "$\hat{t}_{lags}$",
    "dmon_large": "$a_l$",
    "dcrowd": "$a_c$",
    "dcyl": "$D$",
    "nmon_small": "$n_s$",
    "nmon_large": "$n_l$"
}
space_titles = {}
for space in spaces:
    s_info = project_details[project]['parser'](
        space, 
        geometry=geometry,
        group=group,
        lineage='space',
        ispath=False
    )
    space_title_style = {
        'SumRule': fr" $N={s_info.nmon}, D={s_info.dcyl}, a_c={s_info.dcrowd}$",
        'TransFoci': fr" $n_s={s_info.nmon_small}, n_l={s_info.nmon_large}, a_l={s_info.dmon_large}, D={s_info.dcyl}, a_c={s_info.dcrowd}$"
        }
    space_titles[space] = space_title_style[project]

### Some explorations:

In [ ]:
chainsize.head()

In [ ]:
chainsize.columns

In [ ]:
chainsize.info()

### Time series per physical properties per space with an attribute for hues and cols

In [ ]:
hue_attr = 'phi_c_bulk_round'
col_attr = 'phi_c_bulk_round'
x_prop = 'time'
for property_ in properties:
    for space in spaces:
        chainsize_space = chainsize.loc[chainsize['space']==space]
        plotter.p_tseries_space(
            property_,
            hue_attr,
            col_attr,
            chainsize_space,
            space,
            project,
            chainsize_props_specs_project,
            attr_titles,
            x_property= 'time',
            fontsize=20,
            col_wrap= 3
        )

### Time series of each property per porject colored by space col wrapped by attribute

In [ ]:
fontsize=16
x_prop = 'time'
font_scale = 2

col_attr = 'phi_c_bulk_round'
hue_attr = 'dmon_large'
col_wrap = 3
height = 3
aspect = 1.5 * 1.618 # golden ratio
color_palette='colorblind'

for y_prop in properties:
    plotter.p_tseries_allInOne_space(
        chainsize,
        project,
        x_prop,
        y_prop,
        hue_attr,
        col_attr,
        chainsize_props_specs_project,
        attr_titles,
        font_scale=font_scale,
        fontsize=fontsize,
        height=height,
        aspect=aspect,
        col_wrap=col_wrap,
        color_palette=color_palette
    )

col_attr = 'dmon_large'
hue_attr = 'phi_c_bulk_round'
col_wrap = 1
height = 3
aspect = 2 * 1.618 # golden ratio
color_palette='rocket_r'

for y_prop in properties:
    plotter.p_tseries_allInOne_space(
        chainsize,
        project,
        x_prop,
        y_prop,
        hue_attr,
        col_attr,
        chainsize_props_specs_project,
        attr_titles,
        font_scale=font_scale,
        fontsize=fontsize,
        height=height,
        aspect=aspect,
        col_wrap=col_wrap,
        color_palette=color_palette
    )

# Equilbrium size-related measures

#### Extimate equilibrium properties from fitting model

In [ ]:
allInOne_db = '/Users/amirhsi_mini/OneDrive - University of Waterloo/PhD Research/Jupyter/Datasets/'
equil_db = '-'.join(
    ['allInOne', project, group, species, 'equilProps', phase]
)
equil_db = allInOne_db + equil_db + '.csv'
chainsize_equil = pd.read_csv(equil_db)

# selecting 'norm' measures of properties
normalized_props = [prop for prop in chainsize_equil.columns if prop.endswith('-norm')]
attributes = project_details[project]['equil_attributes']
selected_cols = attributes + normalized_props
# melting properties columns
chainsize_melted = chainsize_equil[selected_cols].melt(
    id_vars=attributes,
    value_vars=normalized_props
)

In [ ]:
normalized_props

In [ ]:
norm_properties_specs =  {
    'SumRule': {
        'rfloryMon-norm': {
            'name': 'Flory radius',
            'symbol': r'$\frac{R_F(\phi_c)}{R_{F,0}}$',
            'color': 'firebrick'
        },
        'gyrMon-norm': {
            'name': 'Radius of gyration',
            'symbol': r'$\frac{R_g(\phi_c)}{R_{g,0}}$',
            'color':'steelblue'
        },
        'fsdMon-norm': {
            'name': 'Furthermost distance',
            'symbol': r'$\frac{\fract{L(\phi_c)}{L_{0}}$',
            'color': 'forestgreen'
        },
        'asphericityMon-norm': {
            'name': 'Asphericity',
            'symbol': r'$\frac{\Delta(\phi_c)}{\Delta_{0}}$',
            'color': 'goldenrod'
        },
        'shapeMon-norm': {
            'name': 'Shape parameter',
            'symbol': r'$\frac{S(\phi_c)}{S_{0}}$',
            'color': 'orchid'
        }
    },
    'TransFoci':{
        'gyrMon-norm': {
            'name': 'Radius of gyration',
            'symbol': r'$\frac{R_g(\phi_c)}{R_{g,0}}$',
            'color':'steelblue'
        },
        'fsdMon-norm': {
            'name': 'Furthermost distance',
            'symbol': r'$\frac{L(\phi_c)}{L_{0}}$',
            'color': 'forestgreen'
        },
        'asphericityMon-norm': {
            'name': 'Asphericity',
            'symbol': r'$\frac{\Delta(\phi_c)}{\Delta_{0}}$',
            'color': 'goldenrod'
        },
        'shapeMon-norm': {
            'name': 'Shape parameter',
            'symbol': r'$\frac{S(\phi_c)}{S_{0}}$',
            'color': 'orchid'
        }
    }
}

In [ ]:
fontsize=16
sns.set_context(
            font_scale=2,
            rc={
                'font.family': "Times New Roman",
                'mathtext.default': 'regular',
                "text.usetex": True,
                "font.size": fontsize
            }
        )
tseries_fit_fgrid = sns.relplot(
    x='phi_c_bulk_round',
    y='value',
    col='variable',
    hue='dmon_large',
    kind='line',
    marker="s",
    markersize=7,
    ls=":",
    data=chainsize_melted,
    col_wrap=2,
    facet_kws = {'sharey': False, 'sharex': False}
)
for ax, property_ in zip(tseries_fit_fgrid.axes.flat,normalized_props):
        ax.set_ylabel(
            norm_properties_specs[project][property_]["symbol"],
            rotation=0,
            labelpad=20
        )
        ax.set_title(None)
tseries_fit_fgrid.set_xlabels(attr_titles[hue_attr])
tseries_fit_fgrid.tight_layout(w_pad=0)

In [ ]:
for ax, hue_unique in zip(tseries_fit_fgrid.axes.flat,hue_unique_values):
        long_time_value = np.round(
            chainsize_quilibrium.loc[chainsize_quilibrium[hue_attr]==hue_unique,property_equil_name+"-residue"].values[0],
            2
        )
        #label_value = mean_labels[property_].format(long_time_value)
        ax.axhline(
            long_time_value,
            xmin=0.02,
            xmax=0.98,
            color="black",
            lw=2,
            alpha=0.7,
            label= properties_mon_equil_labels[property_equil_name]['symbol'],
            linestyle="-.")

        handles, labels = ax.get_legend_handles_labels()
        new_labels.append(labels[-1])
        ax.legend(handles = handles, labels=new_labels, frameon=False, ncol=3)
        new_labels = legend_labels[property_]



#### Single equilibrium plots 

In [ ]:
fontsize=16
ext='pdf'
save_to = './'
property_ = 'gyrMon'
s_info = parser(
        space, 
        geometry='biaxial',
        group=group,
        lineage='space',
        ispath=False
    )
space_title =  fr" $N={s_info.nmon}, D={s_info.dcyl}, a_c={s_info.dcrowd}$"
#space_title =  f" $n_s={s_info.nmon_small}, n_l={s_info.nmon_large}, a_l={s_info.dmon_large}, D={s_info.dcyl}, a_c={s_info.dcrowd}$"
for property_ in properties_mon_equil:
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,4.5))
    sns.set_context(
            font_scale=2,
            rc={
                'font.family': "Times New Roman",
                'mathtext.default': 'regular',
                "text.usetex": True,
                "font.size": fontsize
            }
        )
    line_ax = sns.lineplot(
        x=hue_attr,
        y=property_ + "-norm",
        marker="o",
        ls=":",
        data=chainsize_quilibrium,
        ax=ax,
    )
    ax.set_ylabel(
        properties_mon_equil_labels[property_]['norm'],
        rotation=0,
        labelpad=20
    )
    ax.set_xlabel(titles[hue_attr])
    ax.set_title(space_title)
    output = "-".join(["equilPlot", project_name, space,property_,hue_attr]) + "." + ext
    fig.savefig(save_to + output, bbox_inches='tight')
    plt.close()

# Bonds and Clusters 